In [322]:
from bs4 import BeautifulSoup
import requests
import time
from urllib.request import urlopen, Request
from selenium import webdriver

options = webdriver.ChromeOptions()
options.add_argument('--ignore-certificate-errors')
options.add_argument('--incognito')
options.add_argument('--headless')
driver = webdriver.Chrome("/usr/lib/chromium-browser/chromedriver", chrome_options=options)

# f = open("book_titles.txt", "x")
# f2 = open("book_descriptions.txt", "x")
# f3 = open("book_genres.txt", "x")


/var/folders/c7/cnkrdqk12hj7hh7h82qw9xg80000gn/T/ipykernel_2475/2349466462.py:11: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome("/usr/lib/chromium-browser/chromedriver", chrome_options=options)
/var/folders/c7/cnkrdqk12hj7hh7h82qw9xg80000gn/T/ipykernel_2475/2349466462.py:11: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome("/usr/lib/chromium-browser/chromedriver", chrome_options=options)


In [339]:
import urllib.request

headers = {'User-Agent': 'Mozilla/5.0'}

book_titles = []
book_descriptions = []
book_genres = []
book_authors = []

page_link = "https://www.goodreads.com/list/show/4093.Best_Books_of_the_Decade_2010s?page=1"
# driver.get(page_link)
# soup = BeautifulSoup(driver.page_source, 'html.parser')
page = requests.get(page_link, headers=headers)
soup = BeautifulSoup(page.content, 'html.parser')
idx = 0
for i in range(1):
    books = soup.find_all('a', class_='bookTitle')
    
    for i in books:
        link = "https://goodreads.com" + i['href']
        time.sleep(1)
        resp = requests.get(link, headers=headers)

        book_soup = BeautifulSoup(resp.content, 'html.parser')
        # print(book_soup)
        description = book_soup.find(class_="DetailsLayoutRightParagraph__widthConstrained")
        if description == None:
            if book_soup.find(class_="readable stacked") == None:
                print("No description")
                description = "No description"
            else:
                description = book_soup.find(class_="readable stacked").find('span', style="display:none").get_text()
            
        else:
            description = description.text
            
        book_descriptions.append(description)
        
        book_titles.append(book_soup.find("meta", property="og:title").get('content'))
        genre_buttons = []
    
        genre_buttons = book_soup.find_all(class_="actionLinkLite bookPageGenreLink")
        if(len(genre_buttons) == 0):
            genre_buttons = book_soup.find_all(class_='BookPageMetadataSection__genreButton')
    
        single_book_genres = []
        genre_i = 0
        for j in genre_buttons:
            if(genre_i == 5):
                break
            single_book_genres.append(j.text)
            genre_i += 1
        book_genres.append(single_book_genres)
        # if idx == 5:
        #     break
        # idx += 1
       
        author_name = book_soup.find(class_="authorName")
        if author_name == None:
            if book_soup.find(class_="ContributorLink__name") == None:
                author_name = "No author"
                break
            author_name = book_soup.find(class_="ContributorLink__name").get_text()
        else:
            author_name = author_name.get_text()
        book_authors.append(author_name)
        

    page_link = 'https://goodreads.com' + soup.find('a', class_='next_page').get('href')
    # driver.get(page_link)
    # soup = BeautifulSoup(driver.page_source, 'html.parser')
    page = requests.get(page_link, headers=headers)
    soup = BeautifulSoup(page.content, 'html.parser')
    


https://www.goodreads.com/list/show/4093.Best_Books_of_the_Decade_2010s?page=1
<!DOCTYPE html>

<html class="desktop withSiteHeaderTopFullImage">
<head prefix="og: http://ogp.me/ns# fb: http://ogp.me/ns/fb# good_reads: http://ogp.me/ns/fb/good_reads#">
<title>The Fault in Our Stars by John Green</title>
<meta content="The Fault in Our Stars book. Read 169,557 reviews from the world's largest community for readers. Despite the tumor-shrinking medical miracle that has bo..." name="description"/>
<meta content="telephone=no" name="format-detection"/>
<link href="https://www.goodreads.com/book/show/11870085-the-fault-in-our-stars" rel="canonical"/>
<meta content="2415071772" property="fb:app_id"/>
<meta content="books.book" property="og:type"/>
<meta content="The Fault in Our Stars" property="og:title"/>
<meta content="Despite the tumor-shrinking medical miracle that has bought her a few years, Hazel has never been anything but terminal, her final chapte..." property="og:description"/>
<me

In [340]:
# f = open("book_titles.txt", "w")
# f2 = open("book_descriptions.txt", "w")
# f3 = open("book_genres.txt", "w")

# f.write(book_titles)
# f.close()
# f2.write(book_descriptions)
# f2.close()
# f3.write(book_genres)
# f3.close()

print(book_authors)

['John Green', 'Veronica Roth', 'Suzanne Collins', 'Gillian Flynn', 'Andy Weir']
